# .csv to PSQL 
- connect to the postgreSQL with a .gitignore file to hide your login credentials

**How to connect**
 
1) create the initialization file:

file name: 'database_file.ini'
```
[postgresql]
host=localhost
database=what_databse_you_want_to_access
user= some_user_you_created_for_this_user_in_psql
password= some_password_you_have_to_this_db
```

2) create .gitignore file<br>
    ```touch .gitignore```<br>
3) add the database.ini to the .gitignore file<br>
3) create config.py file<br>

In [1]:
import psycopg2             # python->psql connection
import psycopg2.extras
import pandas as pd         # create dataframes 
import os                   # fetch files
from config import config
from functools import wraps # decorator/wrapper
import time                 # timing operations
import memory_profiler      # managing memory usage
from memory_profiler import memory_usage


# Get the config params
params_ = config()
# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)
# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()
conn.autocommit = True

# .csv to psql (COPY function)

In [2]:
def create_staging_table_(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS staging_fake_ppl;
        CREATE UNLOGGED TABLE staging_fake_ppl (
            credit_card         TEXT,
            email               TEXT,
            first_name          TEXT,
            last_name           TEXT,
            primary_phone  TEXT
        );""")
with conn.cursor() as cursor:
    create_staging_table_(cursor)

In [3]:
def send_csv_to_psql(connection,csv,table_):
    sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
    file = open(csv, "r")
    table = table_
    with connection.cursor() as cur:
        cur.execute("truncate " + table + ";")  #avoiding uploading duplicate data!
        cur.copy_expert(sql=sql % table, file=file)
        conn.commit()
#         cur.close()
#         conn.close()
    return conn.commit()

In [4]:
# Remove the first column
addr_df = pd.read_csv('data/user.csv')
addr_df = addr_df.drop('Unnamed: 0',axis=1)
addr_df.to_csv('data/user_cleaned.csv',index=False)

send_csv_to_psql(conn,'data/user_cleaned.csv','staging_fake_ppl')

In [5]:
# Another way of reading table
import pandas.io.sql as sqlio
sql = "select * from staging_fake_ppl limit 4;"
dat = sqlio.read_sql_query(sql, conn)
dat

,credit_card,email,first_name,last_name,primary_phone
0,5399-3484-4724-7187,gso@qiegan.sqe,Donyell Ann,Ospina,5219459148
1,1630-5261-6108-7631,xnji@gfruaxqnvm.fha,Bishop,Siyed,4164254716
2,4435-3866-1076-3595,dvyco@tkzhsop.zxg,Connor,Powers,3627413915
3,3489-7099-9906-8660,fy@uvfhplatmz.cam,Kylie,Her,3562764561


# .csv to psql (INSERT INTO function)

In [6]:
def create_staging_table_(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS staging_fake_ppl02;
        CREATE UNLOGGED TABLE staging_fake_ppl02 (
            credit_card         TEXT,
            email               TEXT,
            first_name          TEXT,
            last_name           TEXT,
            primary_phone  TEXT
        );""")
with conn.cursor() as cursor:
    create_staging_table_(cursor)

In [7]:
def fcn(df,table,cur):
    if len(df) > 0:
        df_columns = list(df)
        # create (col1,col2,...)
        columns = ",".join(df_columns)
        # create VALUES('%s', '%s",...) 
        values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 
        #create INSERT INTO table (columns) VALUES('%s',...)
        insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)
        cur.execute("truncate " + table + ";")  #avoiding uploading duplicate data!
        cur = conn.cursor()
        psycopg2.extras.execute_batch(cur, insert_stmt, df.values)
    conn.commit()

In [8]:
fcn(addr_df,'staging_fake_ppl02',cur)

# Close the cursor and connection to so the server can allocate
# bandwidth to other requests
# cur.close()
# conn.close()